## Naive Linear regression model

In [1]:
import sys
print(sys.executable)

c:\Users\Franck\AppData\Local\Programs\Python\Python38\python.exe


In [2]:
import sys
import os

# Add config_local to Python path so we can import local_config
local_config_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'config_local'))
if local_config_path not in sys.path:
    sys.path.insert(0, local_config_path)

import local_config


In [3]:
import pandas as pd

# Load training and test data using local_config paths
train = pd.read_csv(local_config.TRAIN_CSV)
test = pd.read_csv(local_config.TEST_CSV)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
train.head()

Train shape: (1460, 81)
Test shape: (1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
non_null_poolqc = train['PoolQC'].dropna().unique()
print(non_null_poolqc)

['Ex' 'Fa' 'Gd']


TODO: change missing value in pool aetcc as 0 and 1

In [5]:
missing = train.isnull().sum()
notMissing= missing.loc[missing.values >0].index.tolist()

In [6]:
missing

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [7]:
numerical_cols = train.select_dtypes(include=['number']).columns.tolist()
numerical_cols = [col for col in numerical_cols if col != 'SalePrice']


In [14]:
# For now we pick a few numerical features with low/nulls
features = ['GrLivArea', 'OverallQual', 'YearBuilt','Fence'] #list(set(notMissing) & set(numerical_cols))#
target = 'SalePrice'

X = train[features]
y = train[target]

# Do the same for test set (no target there)
X_test = test[features]

In [15]:
features

['GrLivArea', 'OverallQual', 'YearBuilt', 'Fence']

In [17]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)

print("Model trained!")

ModuleNotFoundError: No module named 'sklearn'

In [15]:
predictions = model.predict(X_test)

# Prepare submission
submission = pd.DataFrame({
    "Id": test["Id"],
    "SalePrice": predictions
})

# Save using the path from local_config
submission_path = os.path.join(local_config.SUBMISSIONS_DIR, "baseline_submission.csv")
submission.to_csv(submission_path, index=False)

submission.head()

,Id,SalePrice
0,1461,109627.820342
1,1462,160206.886968
2,1463,173140.284687
3,1464,197284.356357
4,1465,224560.176828


----

In [8]:
import os, sys
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Add config_local to Python path so we can import local_config
local_config_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'config_local'))
if local_config_path not in sys.path:
    sys.path.insert(0, local_config_path)

import local_config  # expects TRAIN_CSV, TEST_CSV, SUBMISSIONS_DIR

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
assert os.path.exists(local_config.TRAIN_CSV), "TRAIN_CSV not found"
assert os.path.exists(local_config.TEST_CSV),  "TEST_CSV not found"

train = pd.read_csv(local_config.TRAIN_CSV)
test  = pd.read_csv(local_config.TEST_CSV)

print(f"Train shape: {train.shape}  |  Test shape: {test.shape}")
display(train.head(3))

# Ensure target is only in train
assert 'SalePrice' in train.columns and 'SalePrice' not in test.columns, \
    "SalePrice must be present only in train."

In [6]:
# Keep it naïve: a few strong numeric predictors that exist in both sets
candidate_numeric = [
    'GrLivArea', 'OverallQual', 'YearBuilt',
    'GarageCars', 'FullBath', 'TotRmsAbvGrd'
]

# Keep those that actually exist in the current files
features = [c for c in candidate_numeric if c in train.columns and c in test.columns]

X = train[features].copy()
y = train['SalePrice'].copy()
X_test = test[features].copy()

print("Features used:", features)

Features used: ['GrLivArea', 'OverallQual', 'YearBuilt', 'GarageCars', 'FullBath', 'TotRmsAbvGrd']


In [ ]:
model = LinearRegression()
model.fit(X, y)

pred_in = model.predict(X)
rmse = mean_squared_error(y, pred_in, squared=False)
print("Model trained.")
print("Intercept:", model.intercept_)
print("Coefficients:", dict(zip(features, model.coef_)))
print(f"Train RMSE: {rmse:,.0f}")

In [ ]:
predictions = model.predict(X_test)

submission = pd.DataFrame({
    "Id": test["Id"],
    "SalePrice": predictions
})

os.makedirs(local_config.SUBMISSIONS_DIR, exist_ok=True)
submission_path = os.path.join(local_config.SUBMISSIONS_DIR, "baseline_submission.csv")
submission.to_csv(submission_path, index=False)

print(f"Saved submission -> {submission_path}")
display(submission.head())